In [ ]:
# Task 2: Exploratory Data Analysis (EDA) for Bati Bank Credit Risk Model

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [11]:
# Make sure plot directory exists
plot_dir = "../plots/task-2"
os.makedirs(plot_dir, exist_ok=True)

# Load the data
df = pd.read_csv("./../data/raw/data.csv")

# Convert date column to datetime object early on
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

In [12]:
# --- 1. Initial Data Overview ---
print("--- 1. Initial Data Overview ---")
print(f"Dataset Shape: {df.shape}")
print("\nData Types and Non-Null Counts:")
df.info()
print("\nFirst 5 Rows:")
print(df.head())

--- 1. Initial Data Overview ---
Dataset Shape: (95662, 16)

Data Types and Non-Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   TransactionId         95662 non-null  object             
 1   BatchId               95662 non-null  object             
 2   AccountId             95662 non-null  object             
 3   SubscriptionId        95662 non-null  object             
 4   CustomerId            95662 non-null  object             
 5   CurrencyCode          95662 non-null  object             
 6   CountryCode           95662 non-null  int64              
 7   ProviderId            95662 non-null  object             
 8   ProductId             95662 non-null  object             
 9   ProductCategory       95662 non-null  object             
 10  ChannelId             95662 non-null 

In [13]:
# --- 2. Missing Value Analysis (CRITICAL ADDITION) ---
print("\n--- 2. Missing Value Analysis ---")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_values, 'Missing Percentage': missing_percentage})
print(missing_info.sort_values(by='Missing Percentage', ascending=False))

# Visualize missing data
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap')
plt.savefig(f"{plot_dir}/missing_data_heatmap.png")
plt.clf()
# INSIGHT: No missing values were detected in this dataset. If there were, we would need to decide on an imputation strategy (mean, median, mode, or a more complex method) or drop columns/rows.


--- 2. Missing Value Analysis ---
                      Missing Count  Missing Percentage
TransactionId                     0                 0.0
BatchId                           0                 0.0
AccountId                         0                 0.0
SubscriptionId                    0                 0.0
CustomerId                        0                 0.0
CurrencyCode                      0                 0.0
CountryCode                       0                 0.0
ProviderId                        0                 0.0
ProductId                         0                 0.0
ProductCategory                   0                 0.0
ChannelId                         0                 0.0
Amount                            0                 0.0
Value                             0                 0.0
TransactionStartTime              0                 0.0
PricingStrategy                   0                 0.0
FraudResult                       0                 0.0


<Figure size 1200x600 with 0 Axes>

In [14]:
# --- 3. Target Variable: Fraud Distribution ---
print("\n--- 3. Target Variable: Fraud Distribution ---")
fraud_rate = df['FraudResult'].mean()
print(f"Overall Fraud Rate: {fraud_rate:.3%}")
print("\nFraud label distribution (Percentage):")
print(df['FraudResult'].value_counts(normalize=True) * 100)

plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='FraudResult')
plt.title('Fraud vs Non-Fraud Distribution')
plt.savefig(f"{plot_dir}/fraud_distribution.png")
plt.clf()
# INSIGHT: The dataset is extremely imbalanced (0.2% fraud). This will require strategies like over/under-sampling (e.g., SMOTE) or using appropriate evaluation metrics (Precision, Recall, F1-score, AUC-ROC) instead of accuracy.



--- 3. Target Variable: Fraud Distribution ---
Overall Fraud Rate: 0.202%

Fraud label distribution (Percentage):
FraudResult
0    99.798248
1     0.201752
Name: proportion, dtype: float64


<Figure size 600x400 with 0 Axes>

In [15]:
# --- 4. Time-Based Analysis (Enhanced) ---
print("\n--- 4. Time-Based Analysis ---")
df['Hour'] = df['TransactionStartTime'].dt.hour
df['DayOfWeek'] = df['TransactionStartTime'].dt.day_name()
df['Month'] = df['TransactionStartTime'].dt.month

# Plot transaction counts by hour and day of week
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.countplot(data=df, x='Hour', ax=axes[0])
axes[0].set_title('Transaction Count by Hour of Day')
sns.countplot(data=df, x='DayOfWeek', ax=axes[1], order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
axes[1].set_title('Transaction Count by Day of Week')
plt.tight_layout()
plt.savefig(f"{plot_dir}/time_distribution_counts.png")
plt.clf()

# Plot fraud rate by hour and day of week
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.barplot(data=df, x='Hour', y='FraudResult', estimator=np.mean, ax=axes[0])
axes[0].set_title('Mean Fraud Rate by Hour of Day')
axes[0].set_ylabel('Fraud Rate')
sns.barplot(data=df, x='DayOfWeek', y='FraudResult', estimator=np.mean, ax=axes[1], order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
axes[1].set_title('Mean Fraud Rate by Day of Week')
axes[1].set_ylabel('Fraud Rate')
plt.tight_layout()
plt.savefig(f"{plot_dir}/time_distribution_fraud_rate.png")
plt.clf()
# INSIGHT: Transactions (and fraud) seem to peak in the late morning and early evening. Fraud rates appear higher during early morning hours (0-5 AM), a common pattern for fraudulent activities.



--- 4. Time-Based Analysis ---


<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

In [16]:
# --- 5. Numerical Feature Analysis (Enhanced) ---
print("\n--- 5. Numerical Feature Analysis ---")
# Select only true numeric features, excluding IDs and constant columns
numeric_cols = ['Amount', 'Value', 'PricingStrategy']
print("\n🔹 Summary Statistics for Numerical Features:")
print(df[numeric_cols].describe())

# Investigate negative 'Amount' values
negative_amount_count = (df['Amount'] < 0).sum()
print(f"\nFound {negative_amount_count} transactions with negative 'Amount'. These could be reversals or credits.")
print("Transactions with the largest negative amounts:")
print(df[df['Amount'] < 0].sort_values('Amount').head())

# Plot Amount distribution (Original and Log-transformed)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.histplot(df['Amount'], bins=100, kde=False, ax=axes[0])
axes[0].set_title('Transaction Amount Distribution (Original)')
# Use log transform to handle skewness. Add a constant to handle zero and negative values before log.
df['Amount_log'] = np.log1p(df['Amount'] - df['Amount'].min()) 
sns.histplot(df['Amount_log'], bins=100, kde=True, ax=axes[1])
axes[1].set_title('Transaction Amount Distribution (Log-Transformed)')
plt.savefig(f"{plot_dir}/amount_distribution_enhanced.png")
plt.clf()

# Compare transaction amount by fraud (using log scale for y-axis for better visibility)
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='FraudResult', y='Amount')
plt.yscale('symlog') # Use symmetrical log scale to handle negative values
plt.title('Transaction Amount by Fraud Result (Symmetrical Log Scale)')
plt.savefig(f"{plot_dir}/amount_by_fraud_logscale.png")
plt.clf()
# INSIGHT: The 'Amount' is heavily right-skewed. Fraudulent transactions seem to have a wider range and potentially higher median value than non-fraudulent ones, which is more visible on a log scale.


--- 5. Numerical Feature Analysis ---

🔹 Summary Statistics for Numerical Features:
             Amount         Value  PricingStrategy
count  9.566200e+04  9.566200e+04     95662.000000
mean   6.717846e+03  9.900584e+03         2.255974
std    1.233068e+05  1.231221e+05         0.732924
min   -1.000000e+06  2.000000e+00         0.000000
25%   -5.000000e+01  2.750000e+02         2.000000
50%    1.000000e+03  1.000000e+03         2.000000
75%    2.800000e+03  5.000000e+03         2.000000
max    9.880000e+06  9.880000e+06         4.000000

Found 38189 transactions with negative 'Amount'. These could be reversals or credits.
Transactions with the largest negative amounts:
             TransactionId         BatchId       AccountId  \
55757  TransactionId_64133  BatchId_100855  AccountId_4249   
55763  TransactionId_33290   BatchId_33774  AccountId_4249   
55822  TransactionId_56954   BatchId_76447  AccountId_4249   
56035  TransactionId_87682  BatchId_101919  AccountId_4249   
55841  Tran

<Figure size 1600x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [17]:
# --- 6. Categorical Feature Analysis (Enhanced) ---
print("\n--- 6. Categorical Feature Analysis ---")
# Exclude high-cardinality ID columns for general plotting
categorical_cols = ['ProductCategory', 'ChannelId', 'PricingStrategy', 'CurrencyCode']

for col in categorical_cols:
    # Check cardinality
    cardinality = df[col].nunique()
    print(f"Cardinality of {col}: {cardinality}")
    
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, x=col, y='FraudResult', estimator=np.mean, order=df.groupby(col)['FraudResult'].mean().sort_values(ascending=False).index)
    plt.title(f"Fraud Rate by {col}")
    plt.ylabel("Mean Fraud Rate")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f"{plot_dir}/fraud_by_{col}.png")
    plt.clf()
# INSIGHT: 'financial services' product category and 'checkout' channel show significantly higher fraud rates. This is a strong signal for the model.


--- 6. Categorical Feature Analysis ---
Cardinality of ProductCategory: 9
Cardinality of ChannelId: 4
Cardinality of PricingStrategy: 4
Cardinality of CurrencyCode: 1


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [18]:
# --- 7. Correlation Analysis (Refined) ---
print("\n--- 7. Correlation Analysis ---")
# Use the refined list of numeric columns, plus the target variable and time features
correlation_cols = numeric_cols + ['FraudResult', 'Hour']
plt.figure(figsize=(10, 8))
sns.heatmap(df[correlation_cols].corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap of Key Numerical Features")
plt.tight_layout()
plt.savefig(f"{plot_dir}/correlation_heatmap_refined.png")
plt.clf()
# INSIGHT: 'Amount' and 'Value' are perfectly correlated (corr=1.0), which is expected as one is the absolute of the other. We should only keep one of them for modeling to avoid multicollinearity. No other strong linear correlations are immediately obvious.


--- 7. Correlation Analysis ---


<Figure size 1000x800 with 0 Axes>

In [19]:
# --- 8. Summary of Key EDA Insights ---
print("\n--- 8. Summary of Key EDA Insights ---")
insights = """
1.  **Critical Class Imbalance:** Fraudulent transactions make up only 0.2% of the data. This requires careful model selection and evaluation using metrics like F1-Score or AUC, not accuracy. Resampling techniques (like SMOTE) will be necessary during training.
2.  **No Missing Data:** The dataset is complete, simplifying the preprocessing pipeline as no imputation is needed.
3.  **Informative Categorical Features:** `ProductCategory` ('financial services') and `ChannelId` ('checkout') are strong indicators of fraud risk, showing much higher average fraud rates. These features must be encoded and included in the model.
4.  **Data Integrity Issues:** The `Amount` column contains negative values that need to be understood (likely credits/reversals). The `CountryCode` column is constant and provides no information, so it should be dropped. `Amount` and `Value` are redundant.
5.  **Behavioral Patterns:** Fraudulent activities are more prevalent during early morning hours (0-5 AM), suggesting a useful temporal feature. Transaction amounts for fraud cases appear to be higher on average than legitimate ones.
"""
print(insights)


--- 8. Summary of Key EDA Insights ---

1.  **Critical Class Imbalance:** Fraudulent transactions make up only 0.2% of the data. This requires careful model selection and evaluation using metrics like F1-Score or AUC, not accuracy. Resampling techniques (like SMOTE) will be necessary during training.
2.  **No Missing Data:** The dataset is complete, simplifying the preprocessing pipeline as no imputation is needed.
3.  **Informative Categorical Features:** `ProductCategory` ('financial services') and `ChannelId` ('checkout') are strong indicators of fraud risk, showing much higher average fraud rates. These features must be encoded and included in the model.
4.  **Data Integrity Issues:** The `Amount` column contains negative values that need to be understood (likely credits/reversals). The `CountryCode` column is constant and provides no information, so it should be dropped. `Amount` and `Value` are redundant.
5.  **Behavioral Patterns:** Fraudulent activities are more prevalent duri